In [13]:
%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Definición de Librería

In [1]:
import tweepy
import json
import pandas as pd
import time


In [2]:
pd.set_option('display.max_rows', 1000)

In [3]:
#Definción de variables
dataFollowing_id = pd.DataFrame()
dataFollowers_id = pd.DataFrame()

# Definición de las claves para acceder a Twitter

In [4]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [5]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)

In [16]:
#Búsqueda de Tweets por palabra
def busqueda_tweets_palabra(palabra="Ryanair", lenguaje="es", numero_de_Tweets=200):
    return tweepy.Cursor(api.search, palabra, lang=lenguaje).items(numero_de_Tweets)

tweets=busqueda_tweets_palabra()
outtweets = [(tweet.created_at, 
              tweet.text, 
              tweet.retweet_count, 
              str(tweet.favorite_count)+'') for tweet in tweets]
outtweets

[(datetime.datetime(2019, 12, 18, 19, 3, 53),
  'Ryanair atribuye los recortes en Girona a los problemas del Boeing 737 MAX https://t.co/asx2LZmyV3 vía @elpais_economia',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 19, 0, 8),
  'Los asientos separados, el nuevo mercadillo de las low cost (Ryanair, Vueling, etc...) https://t.co/Zg75Rhm4re',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 49, 18),
  'RT @marcosleyley: He visto sardinas en lata que tienen más espacio que volar en Vueling o Ryanair. Es curioso que mientras la población mun…',
  3,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 48, 17),
  'si tienes cualquier problema para que puedas resolverlo "perfectamente" te ponen un número de atención al cliente d… https://t.co/e94lP2nyZ5',
  0,
  '1'),
 (datetime.datetime(2019, 12, 18, 18, 40),
  'viajar con @Ryanair_ES y otras maneras de solo tener problemas.',
  0,
  '2'),
 (datetime.datetime(2019, 12, 18, 18, 36, 2),
  'Inspección de Trabajo abrirá expediente ✈️ https://t.co/

# Extracción de los datos de la cuenta

In [7]:
item = api.get_user('@cabify_espana')
data_account=[]
account={}
account["name"]=item.name
account["id"]=item.id
account["screen name"]=item.screen_name
account["friends_count"]=item.friends_count
account["followers_count"]=item.followers_count
data_account.append(account)
df_account_info = pd.DataFrame(data_account)
df_account_info

,name,id,screen name,friends_count,followers_count
0,Cabify España,369733692,cabify_espana,631,26805


# Extracción de las cuentas Followers

In [8]:
#Nueva forma de obtener todas las cuentas que siguen a Uber/Cabify en España
#cabify_espana
#@Uber_ES
#NokiaMobile
# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_followers_ids(api, twitter_account='@cabify_espana'):
    while True:
        try:
            for page in tweepy.Cursor(api.followers_ids, id=twitter_account, count=5000).pages():
                followers_ids.extend(page)
                #print(len(followers_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return followers_ids

followers_ids=[]
get_followers_ids(api)
#Comprobación de que el número de followers es el correcto
print(len(followers_ids))


26805


In [14]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowers_id= pd.DataFrame(followers_ids)
dataFollowers_id.columns = ['id_Follower']
dataFollowers_id

,id_Follower
0,563473428
1,1223542267660644352
2,221867587
3,4898496711
4,317555938
...,...
26800,306348108
26801,397919847
26802,3054151
26803,356199655


# Extracción de las cuentas Followings

In [10]:
#Nueva forma de obtener todas las cuentas sigue El Corte Inglés/Cabify
#cabify_espana
#elcorteingles

# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_following_ids(api, twitter_account='@cabify_espana'):
    while True:
        try:
            for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
                following_ids.extend(page)
                #print(len(following_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return following_ids

following_ids=[]
get_following_ids(api)
print(len(following_ids))


631


In [15]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowing_id= pd.DataFrame(following_ids)
dataFollowing_id.columns = ['id_Following']
dataFollowing_id


,id_Following
0,48422383
1,1173984355800293377
2,2992924523
3,715535722086801408
4,906103502841290752
5,428136372
6,913184120100081664
7,1076052488921460736
8,2848321193
9,9974322


# Definición de tramos de las cuentas Followers

In [86]:
#Agrupación de ID por la limitación de la API de Twitter
followers_limit=[followers_ids[n:n+99] for n in range(0, len(followers_ids), 100)]
print(len(followers_limit))

269


In [87]:
print(followers_limit)

[[563473428, 1223542267660644352, 221867587, 4898496711, 317555938, 1266795414, 142864449, 1108186707399270400, 1076851198408491008, 3082680657, 1093095183485468673, 773175265891581952, 2854997973, 133433308, 1006229254537990145, 1164276844398555136, 899409043927773184, 1222202681579724800, 707017046, 1222481132576485378, 978689133009887232, 1212819020241522688, 570318157, 176550644, 1979304145, 287708145, 1222188439937916928, 56747456, 1222113635151073280, 1222148308405248000, 198410295, 716574310236872704, 4767144063, 3033240328, 232588983, 1534138338, 759319535958761472, 110727133, 1017087557342023680, 1221490818491539456, 1219581827033305088, 303839590, 1221376852729659392, 1221352137998946305, 2365908040, 3325725645, 1220747190441775104, 186538922, 832170900233584640, 77385383, 2366157368, 968895301514915840, 1054052785493282820, 300416495, 278317782, 1210197127, 210773733, 898274298389028865, 1220487316495577089, 1220266453351510016, 3359130580, 1220187751326330882, 1545630620, 1

# Definición de tramos de las cuentas Following

In [91]:
#Agrupación de ID por la limitación de la API de Twitter
following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
print(len(following_limit))

7


# Extracción de la información de las cuentas Following

In [93]:
# Existe limitación al traer la información de los seguidores
data = []
for followers in followers_limit:
    follower_obj=api.lookup_users(user_ids=followers) 
    for user in follower_obj:
        follower={}
        follower["id"]=user.id
        follower["name"]=user.name
        follower["screen_name"]=user.screen_name
        follower["created_at"]=user.created_at
        follower["location"]=user.location
        follower["followers_count"]=user.followers_count
        follower["friends_count"]=user.friends_count
        follower["description"]=user.description
        data.append(follower)

df_followers_info = pd.DataFrame(data)
df_followers_info

,id,name,screen_name,created_at,location,followers_count,friends_count,description
0,563473428,Rodrigo Espiño,rodri_espino,2012-04-26 03:03:25,De Buenos Aires y Madrid,1756,316,Ex @LATAM_ARG y @Despegar. Amante de los viaje...
1,1223542267660644352,VTC Pro Driver,vtc_driver,2020-02-01 09:43:35,"Madrid, Comunidad de Madrid",2,16,
2,221867587,Gema,GemGM,2010-12-01 20:55:47,,36,81,
3,4898496711,Rute Silva Pereira,rutemcspereira,2016-02-11 16:57:57,"Community of Madrid, Spain",111,111,Portuguese. Foodie. Dog lover. World traveler....
4,317555938,AuTecTic EU,AuTecTic,2011-06-15 03:18:38,127.0.0.1,10559,4612,"About innovation, technology, communication, m..."
...,...,...,...,...,...,...,...,...
26532,306348108,Cárol Antón,carolcosita,2011-05-27 18:31:56,"Madrid, España",178,328,Akamaized girl
26533,397919847,Mª Angeles Buitrón,MangiBuitron,2011-10-25 10:10:01,Madrid,47,53,"Business Analyst, Market Research\nLicenciada ..."
26534,3054151,Joe Fernandez,fdezjose,2007-03-31 10:45:35,"Paris, France",419,401,Enthusiast about the endless possibilities of ...
26535,356199655,Juan de Antonio,jdanru,2011-08-16 14:00:31,Madrid,1313,132,Viajero semi pro y surfer semi amateur. Sin ti...


# Definición de tramos de las cuentas Followers

In [96]:
# Existe limitación al traer la información de los seguidores
data = []
for followings in following_limit:
    following_obj=api.lookup_users(user_ids=followings) 
    for user in following_obj:
        following={}
        following["id"]=user.id
        following["name"]=user.name
        following["screen_name"]=user.screen_name
        following["created_at"]=user.created_at
        following["location"]=user.location
        following["followers_count"]=user.followers_count
        following["friends_count"]=user.friends_count
        following["description"]=user.description
        data.append(following)

df_following_info = pd.DataFrame(data)
df_following_info

,id,name,screen_name,created_at,location,followers_count,friends_count,description
0,48422383,Jonathan Chacón,jonathanchacon,2009-06-18 17:29:37,Transitando por el 🌍,2470,234,Senior accessibility software engineer at @Cab...
1,1173984355800293377,Mobilidade Coruñesa,MobilidadeC,2019-09-17 15:37:47,,1,6,"Ideas, proxectos, actualidade e noticias sobre..."
2,2992924523,Cabify,cabify,2015-01-22 20:30:57,Global,2169,26,¡Bienvenido a Cabify! Si tienes cualquier duda...
3,715535722086801408,Mutua Madrileña,la_Mutua,2016-03-31 13:46:24,,4881,316,#JuntosEvolucionamos hacia todo lo que está po...
4,906103502841290752,Movilidad Sostenible,MadridSummitMS,2017-09-08 10:34:44,"Madrid, Comunidad de Madrid",563,275,Impulsando la Movilidad Sostenible desde las e...
5,428136372,Luis Benavides,LluisBenavides,2011-12-04 11:35:52,"Barcelona, Spain",867,1655,El Periódico de Catalunya. Cuenta personal
6,913184120100081664,La Nave,LaNaveMadrid,2017-09-27 23:30:35,"Madrid, Spain",4677,822,"#LaNave, un espacio de innovación urbana para ..."
7,1076052488921460736,Madrid in Motion,MadridMotion,2018-12-21 09:51:20,,503,322,El hub de movilidad urbana que genera solucion...
8,2848321193,Coca-Cola Comunica,CocaColaCo_es,2014-10-28 15:27:43,"Madrid, España",17452,2028,No vamos a parar hasta conseguirlo. HAGÁMOSLO ...
9,9974322,MΛRC VIDΛL,marcvidal,2007-11-05 18:27:40,Madrid,67641,564,"Speaker en https://t.co/SwiapXx0E2, co-fundé @..."


In [18]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file)

saveID(dataFollowing_id,'followings_id.csv')
#saveID(dataFollowers_id,'followers_id.csv')

In [19]:
#Guardado en ficheros de los IDS de Twitter
def saveInfo(dataframe,file):
    dataframe.to_csv(file)

saveInfo(df_following_info,'followings_info.csv')


In [65]:
for item in df_following_info["screen_name"]:
    print(str(item).replace(' ',''))

jonathanchacon
MobilidadeC
cabify
la_Mutua
MadridSummitMS
LluisBenavides
LaNaveMadrid
MadridMotion
CocaColaCo_es
marcvidal
unicomplutense
eBayESP
burn_es
VectorRonda
marta_plana
elpais_retina
NewEconomyForum
aeropuerto_LCG
WayraES
SemanaRSCMadrid
Ismarbar
AECID_es
davidperezpi
Isabelgarro
lycompany1
albertogil1980
ironhack
TetraPak_es
braysefe
IEbusiness
south_summit
IreneRecio
pabloviewpoint
DGTes
SEUMov
CarlosMarco
DanielGrao7
ivanferreiro
LopezAquinoL
ItziarCastro
yas_salem
esperanzagracia
MAPFRE
moniquecestmoi
unautovtc
movo_esp
PODEMOS
FAMMA_Co_Madrid
Toni_Rami
david_bel
borjaberzosa
carofigoli
jdalamo63
Goorgo
Nacho_Palomar
imartalamas
_PacoCampos
visualmethod
alexlamatrainer
AlbertoCastilla
Emma_Lebon
ANavarreteBueno
vegetta777
frnperea
Avaloncine
carlessanchez
Dahudi92
jcompan
erlantzplaza
luciabrotons
uyalbert
nspmediatv
Kobazuloak
MarcoGinja22
alivcamus
pedroorellanali
AlaineGonzalez
mariajo_suarez
modesto_garcia
Desaze
Wismichu
Joel_Fortuny
garciamadrid
albertochicote
grefus

# Extracción de la información de los Tweets de Followers

In [72]:
#Extracción de los tweets de las cuentas de followers
data = []
for users in df_followers_info["screen_name"]:
    formato_user=str(users).replace(' ','')
    try:   
        tweets_user=api.user_timeline(screen_name=formato_user, count=1)
        for tweet in tweets_user:
            #Creación de diccionarios
            tweets={}
            tweets["id_tweet"]=tweet.id
            tweets["id_user"]=tweet.user.id
            tweets["text"]=tweet.text
            tweets["location"] = tweet.user.location
            tweets["created_at"]=tweet.created_at
            data.append(tweets)
    except tweepy.TweepError:
        print("Fallo por el usuario, ",tweet.user.id," Skipping...")
        pass
            
df_followers_info_tweets = pd.DataFrame(data)
df_followers_info_tweets

Rate limit reached. Sleeping for: 279


KeyboardInterrupt: 

# Extracción de la información de los Tweets de Followings

In [97]:
#Extracción de los tweets de las cuentas de followers
data = []
for users in df_following_info["screen_name"]:
    formato_user=str(users).replace(' ','')
    try:   
        tweets_user=api.user_timeline(screen_name=formato_user, count=1)
        for tweet in tweets_user:
            #Creación de diccionarios
            tweets={}
            tweets["id_tweet"]=tweet.id
            tweets["id_user"]=tweet.user.id
            tweets["text"]=tweet.text
            tweets["location"] = tweet.user.location
            tweets["created_at"]=tweet.created_at
            data.append(tweets)
    except tweepy.TweepError:
        print("Fallo por el usuario, ",tweet.user.id," Skipping...")
        pass
            
df_following_info_tweets = pd.DataFrame(data)
df_following_info_tweets

Fallo por el usuario,  867615151951925248  Skipping...
Fallo por el usuario,  105895230  Skipping...
Fallo por el usuario,  816178  Skipping...


Rate limit reached. Sleeping for: 630


KeyboardInterrupt: 

In [35]:
#Guardado en ficheros de los tweets por usuario de Twitter
def saveTweets(dataframe,file):
    dataframe.to_csv(file)

saveTweets(df_following_info_tweets,'followings_info_tweets.csv')

NameError: name 'df_following_info_tweets' is not defined